## Рекомендательные системы
### Урок 6. Двухуровневые модели рекомендаций

### Для начала возьмём подготовительную часть из урока

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares

# Модель второго уровня
from catboost import CatBoostClassifier

import sys
sys.path.append('../')

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

/home/eugene/study/gb.recom/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [4]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [5]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [7]:
data_train_matcher.head()

,Unnamed: 0,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1832874,1078,35573861879,524,1082185,1,0.56,375,0.0,1440,76,0.0,0.0
1,402281,324,29170411703,165,7168774,2,6.98,367,0.0,1115,24,0.0,0.0
2,1348564,1982,32957769022,404,12811490,1,3.99,319,0.0,2101,58,0.0,0.0
3,1714815,1023,34573871336,495,920025,1,5.99,299,0.0,1643,71,0.0,0.0
4,1266182,695,32672141822,383,941357,1,3.19,396,0.0,1743,55,0.0,0.0


In [8]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 33411 to 5001


In [9]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

In [10]:
recommender = MainRecommender(data_train_matcher)

100%|██████████| 2431/2431 [00:00<00:00, 55215.71it/s]


In [11]:
ACTUAL_COL = 'actual'

In [12]:
result_eval_matcher = data_val_matcher.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head()

,user_id,actual
0,1501,"[8090657, 1016966, 860776, 1119051, 6904428, 8..."
1,1633,"[953476, 916122, 1083043, 9837399, 903524, 983..."
2,336,"[824663, 1096317, 1015375, 1107661, 835300, 10..."
3,2195,"[916260, 896444, 946396, 995965]"
4,2107,"[916122, 1024731, 1070428, 1068504, 849843, 64..."


In [13]:
def calc_precision(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_recall(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### **Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [14]:
def make_recommendations(df_result, name, model, N=50):
    df_result[name] = df_result[USER_COL].apply(lambda x: model(x, N=N))
    
def fill_with_tops(column, N=5):    
    tops = np.array(recommender.overall_top_purchases)
    recs = np.array(column)
    mask = np.isin(tops, recs, invert=True)
    tops = tops[mask]
    return np.append(recs, tops[:N])

In [15]:
rec_list = (
    ('own_recs', recommender.get_own_recommendations, 50),
    ('als_recs', recommender.get_als_recommendations, 50),
    ('similar_user_recs', recommender.get_similar_users_recommendation, 50),
    ('similar_item_recs', recommender.get_similar_items_recommendation, 50),
    ('own+top_pop', recommender.get_own_recommendations, 25)
)

In [ ]:
for name, model, n in rec_list:
    make_recommendations(result_eval_matcher, name, model, n)

In [ ]:
result_eval_matcher['own+top_pop'] = result_eval_matcher['own+top_pop']. \
        apply(lambda row: fill_with_tops(row, N=25))

In [ ]:
sorted(calc_recall(result_eval_matcher, 50), key=lambda x: x[1], reverse=True)

In [ ]:
k_list = (20, 50, 100, 200, 500)
BEST_RECOM = 'own_recs'

scores = [result_eval_matcher.apply(lambda row: recall_at_k(row[BEST_RECOM], row[ACTUAL_COL], k=k), axis=1).mean() for k in k_list]


plt.plot(k_list, scores)
plt.xlabel('k')
plt.ylabel('Recall@k')
plt.grid()
plt.show()

#### Выводы
3 часа работало и ничего... Но в любом случае лучшие показатели будут у recall@50 будут у own_rec. А при увелечении k будет расти и recall@k

---

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()

In [ ]:
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=50))

# Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)
df_ranker_train.target.value_counts()

In [ ]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [ ]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')
cat_feats

In [ ]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [ ]:
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [ ]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]

In [ ]:
result_eval_ranker['own_rec'] = \
result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [ ]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].\
sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [ ]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [ ]:
print(*sorted(calc_precision(result_eval_ranker, 5), key=lambda x: x[1], reverse=True), sep='\n')